- General Setup:
    - 175 grid points
    - Cost function: $c(x, \theta_1) = 0.001 \theta_{11} x$
    - **Missing** true parameters: $RC=11.7257$, $\theta^0_{11}=2.4569$, $\theta^0_3=(0.0937, 0.4475, 0.4459, 0.0127, 0.0002)$, $\beta\in \{0.975, 0.985, 0.995, 0.999, 0.9995, 0.9999\}$
    - simulate $T=120$ (time periods) and $M=50$ (buses)
    - Monte Carlo with 250 data sets per different $\beta$
    - **Missing** Run NFXP and MPEC with the same varying starting values: $(RC^0=4, \theta^0_1=1), (RC^0=5, \theta^0_1=2), (RC^0=6, \theta^0_1=3), (RC^0=7, \theta^0_1=4), (RC^0=8, \theta^0_1=5)$ and in case of MPEC with the starting values $EV_1,...,EV_{75}=(0,...,0)$.
    - **Missing** Give out a table with the mean estimated parameters, their standard deviation and the MSE per $\beta$ and per method 
    - **Missing** Second table with the number of convergences, mean CPU time in seconds, mean number of major iterations, mean number of function evaluations, mean number of Bellman iterations, mean number of N-K iterations per $\beta$ and per method
    
    
- Specifications for NFXP:
    - Stopping tolerance of fixed point algorithm $10^{-13}$
    - Relative tolerance from contraction mapping to NK of $0.02$ and absolute of $0.01$
    - BHHH tolerance of $0.5$ $\rightarrow$ our tolerance is set up differently
    - Maximum number of steps in both contraction mapping and NK is $20$
    - Switching seems possible
    
    
- Specifications for MPEC:
    - constraint tolerance $10^{-6}$, relative tolerance on $x$ is $10^{-15}$ and the relative tolerance on the function values is $10^{-6}$.
    
    
- Differences:
    - We only run NFXP with partial likelihood approach. Not like in their paper where they first caluclate the partial likelihood and then the full again after.
    - They run the transition prob estimation with staring values based on frequency. We do not do that because the transition prob estimation is exactly the same for both NFXP and MPEC.
    - MPEC for them uses sparsity patterns of Hessian and Jacobian as well as analytical Hessian.

In [1]:
import numpy as np
import pandas as pd
import scipy.io
import time

from ruspy.estimation.estimation import estimate
from promotion.replication.auxiliary_iskhakov_et_al_2016.auxiliary import (process_data, process_result)

In [5]:
# Initialize the simulation
discount_factor = [0.975, 0.985, 0.995, 0.999, 0.9995, 0.9999]
approach = ["NFXP", "MPEC"]
starting_cost_params = np.vstack((np.arange(4,9), np.arange(1,6)))
starting_expected_value_fun = np.zeros(175)
number_runs = 10
number_buses = 50
number_periods = 120
number_states = 175
number_cost_params = 2

# Initialize the set up for the nested fixed point algorithm
stopping_crit_fixed_point = 1e-13
switch_tolerance_fixed_point = 1e-2

# Initialize the set up for MPEC
lower_bound = np.concatenate((np.full(number_states, -np.inf), np.full(number_cost_params, 0.0)))
upper_bound = np.concatenate((np.full(number_states, 50.0), np.full(number_cost_params, np.inf)))
rel_func_value_stopping_criteria = 1e-6
rel_x_value_stopping_criteria = 1e-15 

init_dict_nfxp = {
    "model_specifications": {
        "number_states": number_states,
        "maint_cost_func": "linear",
        "cost_scale": 1e-3,
    },
    "optimizer": {
        "approach": "NFXP",
        "algorithm": "estimagic_bhhh",
        "gradient": "Yes",
    },
    "alg_details": {
        "threshold": stopping_crit_fixed_point,
        "switch_tol": switch_tolerance_fixed_point,
    },
}

init_dict_mpec = {
    "model_specifications": {
        "number_states": number_states,
        "maint_cost_func": "linear",
        "cost_scale": 1e-3,
    },
    "optimizer": {
        "approach": "MPEC",
        "algorithm": "LD_SLSQP",
        "gradient": "Yes",
        "set_ftol_rel": rel_func_value_stopping_criteria,
        "set_xtol_rel": rel_x_value_stopping_criteria,
        "set_lower_bounds": lower_bound,
        "set_upper_bounds": upper_bound,
    },
}

In [6]:
# Initialize DataFrame to store the results of each run
index = pd.MultiIndex.from_product([discount_factor, 
                                    range(number_runs),
                                    range(starting_cost_params.shape[1]),
                                    approach],
                                   names=["disc_factor", "run", "start", "approach"])

columns=["RC", "theta_11", "theta_30", "theta_31", "theta_32", "theta_33",
         "CPU Time", "Converged", "# of Major Iter.", "# of Func. Eval.", 
         "# of Bellm. Iter.", "# of N-K Iter."]

results = pd.DataFrame(index=index, columns=columns)

In [7]:
# Main loop to calculate the results for each run
for factor in discount_factor:
    # load simulated data 
    mat = scipy.io.loadmat("auxiliary_iskhakov_et_al_2016/RustBusTableXSimDataMC250_beta" + str(int(100000*factor)))
    
    for run in range(number_runs):
        data = process_data(mat, run, number_buses, number_periods)
        
        for start in range(starting_cost_params.shape[1]):
            # Adapt the Initiation Dictionairy of NFXP for this run
            init_dict_nfxp["model_specifications"]["discount_factor"] = factor
            init_dict_nfxp["optimizer"]["params"] = pd.DataFrame(starting_cost_params[:, start], columns=["value"])
            
            # Run NFXP
            tic = time.perf_counter()
            transition_result_nfxp, cost_result_nfxp = estimate(init_dict_nfxp, data)
            toc = time.perf_counter()
            time_nfxp = toc - tic
            
            results.loc[factor, run, start, "NFXP"] = process_result(
                "NFXP", transition_result_nfxp, cost_result_nfxp, time_nfxp, number_states)
            
            # Adapt the Initiation Dictionairy of MPEC for this run
            init_dict_mpec["model_specifications"]["discount_factor"] = factor
            init_dict_mpec["optimizer"]["params"] = np.concatenate((
                starting_expected_value_fun, starting_cost_params[:, start]))
            
            # Run MPEC
            tic = time.perf_counter()
            transition_result_mpec, cost_result_mpec = estimate(init_dict_mpec, data)
            toc = time.perf_counter()
            time_mpec = toc - tic
            
            results.loc[
                factor, run, start, "MPEC"].loc[
                ~results.columns.isin(["# of Bellm. Iter.", "# of N-K Iter."])] = process_result(
                        "MPEC", transition_result_mpec, cost_result_mpec, time_mpec, number_states)

In [85]:
results

RC  theta_11   theta_30  theta_31  \
disc_factor run start approach                                           
0.9750      0   0     NFXP      8.89422   1.55117  0.0988235  0.442857   
                      MPEC      8.89555   1.55178  0.0988235  0.442857   
                1     NFXP      8.89422   1.55117  0.0988235  0.442857   
                      MPEC      8.89429   1.55122  0.0988235  0.442857   
                2     NFXP      8.89422   1.55117  0.0988235  0.442857   
...                                 ...       ...        ...       ...   
0.9999      9   2     MPEC      4.66564  0.108228  0.0986555  0.439328   
                3     NFXP      10.6258   2.14888  0.0986555  0.439328   
                      MPEC      4.66588  0.108185  0.0986555  0.439328   
                4     NFXP      10.6258   2.14888  0.0986555  0.439328   
                      MPEC      4.66563  0.108231  0.0986555  0.439328   

                                theta_32   theta_33  CPU Time Converged  \
disc_factor run start approach                                            
0.9750      0   0     NFXP      0.446555  0.0112605  0.671914         1   
                      MPEC      0.446555  0.0112605  0.645444         1   
                1     NFXP      0.446555  0.0112605  0.720585         1   
                      MPEC      0.446555  0.0112605   0.83569         1   
                2     NFXP      0.446555  0.0112605  0.769864         1   
...                                  ...        ...       ...       ...   
0.9999      9   2     MPEC      0.449412  0.0122689   0.59483         1   
                3     NFXP      0.449412  0.0122689   1.74026         1   
                      MPEC      0.449412  0.0122689  0.711188         1   
                4     NFXP      0.449412  0.0122689    1.9959         1   
                      MPEC      0.449412  0.0122689  0.738535         1   

                               # of Major Iter. # of Func. Eval.  \
disc_factor run start approach                                     
0.9750      0   0     NFXP                    7                8   
                      MPEC                   11               11   
                1     NFXP                    7                8   
                      MPEC                   13               13   
                2     NFXP                    7                8   
...                                         ...              ...   
0.9999      9   2     MPEC                   10               10   
                3     NFXP                    7                9   
                      MPEC                   14               14   
                4     NFXP                    8               10   
                      MPEC                   16               16   

                               # of Bellm. Iter. # of N-K Iter.  
disc_factor run start approach                                   
0.9750      0   0     NFXP                   220             67  
                      MPEC                   NaN            NaN  
                1     NFXP                   180             57  
                      MPEC                   NaN            NaN  
                2     NFXP                   180             57  
...                                          ...            ...  
0.9999      9   2     MPEC                   NaN            NaN  
                3     NFXP                   200            200  
                      MPEC                   NaN            NaN  
                4     NFXP                   220            220  
                      MPEC                   NaN            NaN  

[600 rows x 12 columns]

In [86]:
# Create Table I from Su & Judd (2012)
columns_table_1 = ["RC", "theta_11", "theta_30", "theta_31", "theta_32", "theta_33"]
table_1 = results[columns_table_1].astype(float)
table_1.groupby(level=["disc_factor", "approach"]).mean()

RC  theta_11  theta_30  theta_31  theta_32  \
disc_factor approach                                                      
0.9750      MPEC      11.489850  2.418784  0.095395  0.445277  0.446454   
            NFXP      11.487675  2.418052  0.095395  0.445277  0.446454   
0.9850      MPEC      12.071581  2.548062  0.095395  0.445277  0.446454   
            NFXP      12.069400  2.547275  0.095395  0.445277  0.446454   
0.9950      MPEC      11.824718  2.442304  0.095395  0.445277  0.446454   
            NFXP      11.823290  2.441796  0.095395  0.445277  0.446454   
0.9990      MPEC      11.969168  2.529202  0.095395  0.445277  0.446454   
            NFXP      11.968418  2.528827  0.095395  0.445277  0.446454   
0.9995      MPEC      12.083115  2.574995  0.095395  0.445277  0.446454   
            NFXP      12.082397  2.574586  0.095395  0.445277  0.446454   
0.9999      MPEC       4.672118  0.107228  0.095395  0.445277  0.446454   
            NFXP      11.933600  2.522335  0.095395  0.445277  0.446454   

                      theta_33  
disc_factor approach            
0.9750      MPEC      0.012655  
            NFXP      0.012655  
0.9850      MPEC      0.012655  
            NFXP      0.012655  
0.9950      MPEC      0.012655  
            NFXP      0.012655  
0.9990      MPEC      0.012655  
            NFXP      0.012655  
0.9995      MPEC      0.012655  
            NFXP      0.012655  
0.9999      MPEC      0.012655  
            NFXP      0.012655

In [80]:
# Create Table I from Iskhakov et al. (2016)
columns_table_2 = ["CPU Time", "Converged", "# of Major Iter.", "# of Func. Eval.", "# of Bellm. Iter.", "# of N-K Iter."]
table_2 = results[columns_table_2].astype(float)
table_2 = table_2.groupby(["disc_factor", "approach"]).mean()
table_2["Converged"] = (table_2["Converged"]*number_runs*starting_cost_params.shape[1]).astype(int)
table_2

CPU Time  Converged  # of Major Iter.  # of Func. Eval.  \
disc_factor approach                                                            
0.9750      MPEC      0.973174         10             17.38             17.38   
            NFXP      1.089977         10              9.98             12.18   
0.9850      MPEC      1.088637         10             19.24             19.24   
            NFXP      1.253559         10             11.24             13.54   
0.9950      MPEC      1.247571         10             22.34             22.34   
            NFXP      1.206287         10             10.74             12.86   
0.9990      MPEC      1.551047         10             26.98             26.98   
            NFXP      1.853100         10             11.84             13.88   
0.9995      MPEC      1.623201         10             28.84             28.84   
            NFXP      2.475205         10             11.08             13.14   
0.9999      MPEC      0.565566         10             10.68             10.68   
            NFXP      2.591928         10             11.14             13.00   

                      # of Bellm. Iter.  # of N-K Iter.  
disc_factor approach                                     
0.9750      MPEC                    NaN             NaN  
            NFXP                 262.88           88.42  
0.9850      MPEC                    NaN             NaN  
            NFXP                 290.42          103.70  
0.9950      MPEC                    NaN             NaN  
            NFXP                 275.68          101.44  
0.9990      MPEC                    NaN             NaN  
            NFXP                 297.22          168.80  
0.9995      MPEC                    NaN             NaN  
            NFXP                 282.04          255.44  
0.9999      MPEC                    NaN             NaN  
            NFXP                 279.24          273.06